In [1]:
# Core
import numpy as np
import pandas as pd
from scipy.interpolate import PchipInterpolator
from scipy.stats import norm
from statsmodels.distributions.empirical_distribution import ECDF

# Machine learning
import tensorflow as tf

# Plotting
import matplotlib.pyplot as plt
import matplotlib as mpl

# Utility
import os
from datetime import datetime

# MSE imports
import kepler_sieve
from asteroid_element import load_ast_elt
from candidate_element import perturb_elts
from asteroid_model import make_model_ast_pos
from nearest_element import nearest_ast_elt, plot_elt_transform_pdf, plot_elt_transform_map
from nearest_element import calc_elt_pos
from nearest_element import ast_elt_transform, calc_beta, make_interp_x
from astro_utils import datetime_to_mjd

Found 4 GPUs.  Setting memory growth = True.


In [2]:
# Set plot style variables
mpl.rcParams['figure.figsize'] = [16.0, 10.0]
mpl.rcParams['font.size'] = 16

In [3]:
# Load orbital elements for known asteroids
ast_elt = load_ast_elt()

# Number of asteroids
N_ast = ast_elt.shape[0]

In [4]:
# Set number of sample points
N_samp_u: int = 2**16
z_range: float = 6.0
N_samp_z: int = int(200*z_range + 1)

# Sample CDF levels: N_samp evenly spaced
cdf_samp_u = (np.arange(N_samp_u) + 0.5) / N_samp_u

# Sample CDF levels: z_samp points evenly spaced by Z
z_samp = np.linspace(-z_range, z_range, N_samp_z)
cdf_samp_z = norm.cdf(z_samp)
pdf_samp_z = norm.pdf(z_samp)

# Combine the two sets of sample points
cdf_samp = np.unique(np.hstack([cdf_samp_u, cdf_samp_z]))

In [5]:
# Review orbital elements
ast_elt

,Num,Name,epoch_mjd,a,e,inc,Omega,omega,M,H,G,Ref,f,P,n,long,theta,pomega,T_peri
Num,,,,,,,,,,,,,,,,,,,
1,1,Ceres,58600.0,2.769165,0.076009,0.184901,1.401596,1.284522,1.350398,3.34,0.12,JPL 46,1.501306,1683.145749,0.003733,4.036516,4.187424,2.686118,-361.745873
2,2,Pallas,58600.0,2.772466,0.230337,0.608007,3.020817,5.411373,1.041946,4.13,0.11,JPL 35,1.490912,1686.155979,0.003726,3.190951,3.639917,2.149005,-279.616804
3,3,Juno,58600.0,2.669150,0.256942,0.226699,2.964490,4.330836,0.609557,5.33,0.32,JPL 108,0.996719,1592.787270,0.003945,1.621697,2.008860,1.012141,-154.522558
4,4,Vesta,58600.0,2.361418,0.088721,0.124647,1.811840,2.630709,1.673106,3.20,0.32,JPL 34,-4.436417,1325.432768,0.004740,6.115656,0.006132,4.442550,-352.940421
5,5,Astraea,58600.0,2.574249,0.191095,0.093672,2.470978,6.260280,4.928221,6.85,0.15,JPL 108,-1.738676,1508.600442,0.004165,1.093108,0.709396,2.448072,325.328481
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1255499,1255499,2019 QG,58600.0,0.822197,0.237862,0.220677,5.066979,3.770460,0.503214,21.55,0.15,JPL 1,0.807024,272.309165,0.023074,-3.225717,-2.921908,-3.728932,-21.808984
1255501,1255501,2019 QL,58600.0,2.722045,0.530676,0.113833,4.741919,2.351059,5.297173,19.21,0.15,JPL 1,-2.082964,1640.368337,0.003830,-0.176219,-1.273172,0.809793,257.420824
1255502,1255502,2019 QQ,58600.0,1.053137,0.389091,0.172121,5.648270,2.028352,3.266522,25.31,0.15,JPL 1,-3.081905,394.753268,0.015917,-1.623227,-1.688469,1.393436,189.527723


## Build and Plot Transformed Orbital Elements

In [ ]:
# Build transformed elements and interpolators
ast_elt_xf, interp_tbl = ast_elt_transform(ast_elt)

In [ ]:
# Review orbital elements with extra transformed columns
ast_elt_xf

## Semimajor Axis, a

In [ ]:
# Plot PDF of a
plot_elt_transform_pdf(ast_elt_xf=ast_elt_xf, elt_name='log_a')

In [ ]:
# Plot transform of a
plot_elt_transform_map(ast_elt_xf=ast_elt_xf, elt_name='log_a')

## Eccentricity, e

In [ ]:
# Plot PDF of e
plot_elt_transform_pdf(ast_elt_xf=ast_elt_xf, elt_name='e')

In [ ]:
# Plot transform of e
plot_elt_transform_map(ast_elt_xf=ast_elt_xf, elt_name='e')

## Inclination, inc

In [ ]:
# Plot PDF of sin_inc
plot_elt_transform_pdf(ast_elt_xf=ast_elt_xf, elt_name='sin_inc')

In [ ]:
# Plot transform of sin_inc
plot_elt_transform_map(ast_elt_xf=ast_elt_xf, elt_name='sin_inc')

## Omega

In [ ]:
# Plot PDF of sin_Omega
plot_elt_transform_pdf(ast_elt_xf=ast_elt_xf, elt_name='sin_Omega')

In [ ]:
# Plot PDF of sin_Omega
plot_elt_transform_map(ast_elt_xf=ast_elt_xf, elt_name='sin_Omega')

In [ ]:
# Plot PDF of cos_Omega
plot_elt_transform_pdf(ast_elt_xf=ast_elt_xf, elt_name='cos_Omega')

In [ ]:
# Plot PDF of cos_Omega
plot_elt_transform_map(ast_elt_xf=ast_elt_xf, elt_name='cos_Omega')

## omega

In [ ]:
# Plot PDF of sin_omega
plot_elt_transform_pdf(ast_elt_xf=ast_elt_xf, elt_name='sin_omega')

In [ ]:
# Plot map of sin_omega
plot_elt_transform_map(ast_elt_xf=ast_elt_xf, elt_name='sin_omega')

In [ ]:
# Plot PDF of cos_omega
plot_elt_transform_pdf(ast_elt_xf=ast_elt_xf, elt_name='cos_omega')

In [ ]:
# Plot PDF of sin_Omega
plot_elt_transform_map(ast_elt_xf=ast_elt_xf, elt_name='cos_omega')

## True Anomaly, f

In [ ]:
# Plot PDF of sin_f
plot_elt_transform_pdf(ast_elt_xf=ast_elt_xf, elt_name='sin_f')

In [ ]:
# Plot map of sin_omega
plot_elt_transform_map(ast_elt_xf=ast_elt_xf, elt_name='sin_f')

In [ ]:
# Plot PDF of cos_f
plot_elt_transform_pdf(ast_elt_xf=ast_elt_xf, elt_name='cos_f')

In [ ]:
# Plot PDF of cos_f
plot_elt_transform_pdf(ast_elt_xf=ast_elt_xf, elt_name='cos_f')

## Covariance of Transformed Elements

In [ ]:
# Build tranformation matrix beta and X_beta for computing distance to orbital elements
beta, X_beta = calc_beta(ast_elt_xf)

In [ ]:
# Assemble covaraiance matrix
Q = np.cov(X_beta, rowvar=False)

In [ ]:
# Demonstrate that Q is the 9x9 identity matrix
Q_err = np.linalg.norm(Q - np.identity(9))
print(f'Error of matrix Q vs. identity: {Q_err:6.2e}')

## Search for Asteroid with Nearest Orbital Elements Based on Covariance of Transformed Elements

In [27]:
# First 32 odd numbered asteroids (don't want consecutive to disambiguate rows and columns)
elt = ast_elt.iloc[0:64:2].copy()

In [ ]:
# Search for near elements on unperturbed asteroids
ast_elt_near = nearest_ast_elt(elt)

In [ ]:
# Review near elements - should be exact matches with 0 distance
ast_elt_near.iloc[0:10]

In [ ]:
# Inputs to perturb elements: large
sigma_a = 0.01
sigma_e = 0.002
sigma_inc_deg = 0.25
sigma_f_deg = 0.5
sigma_Omega_deg = 0.5
sigma_omega_deg = 0.5
mask_pert = None
random_seed = 42

# Perturb orbital elements
elts_pert = perturb_elts(elt, sigma_a=sigma_a, sigma_e=sigma_e, 
                    sigma_inc_deg=sigma_inc_deg, sigma_f_deg=sigma_f_deg, 
                    sigma_Omega_deg=sigma_Omega_deg, sigma_omega_deg=sigma_omega_deg,
                    mask_pert=mask_pert, random_seed=random_seed)

In [ ]:
# Search for near elements on perturbed asteroids
ast_elt_near_pert = nearest_ast_elt(elts_pert)

In [ ]:
# Review near elements - should be close but not zero
ast_elt_near_pert

## Search for Asteroid with Nearest Elements Based on Orbital Trajectories

In [6]:
# Set time array
t0 = datetime_to_mjd(datetime(2018,6,30))
t1 = datetime_to_mjd(datetime(2020,3,31))
dt = 7
ts = np.arange(t0, t1, dt)

In [7]:
elt_ast = ast_elt.iloc[0:64]

In [8]:
q_ast = calc_elt_pos(elt=elt_ast, ts=ts)

In [9]:
q_ast.shape

(64, 92, 3)

In [35]:
N_ast = ast_elt.shape[0]

In [41]:
N_elt = elt.shape[0]

In [36]:
N_t = ts.size

In [42]:
(N_ast, N_elt, N_t, space_dims)

(733489, 32, 92, 3)

In [25]:
# Load the known asteroid positions
q_ast = load_known_ast_pos()

In [ ]:
# Convert to a float32 tensor
X = tf.constant(q_ast, dtype=tf.float32)

In [28]:
# Calculate position of element
q_elt = calc_elt_pos(elt, ts)
# Convert to a tensor; use float32 to save memory
Y = tf.constant(q_elt, dtype=tf.float32)

In [51]:
X.shape

TensorShape([733489, 92, 3])

In [52]:
Y.shape

TensorShape([32, 92, 3])

In [64]:
dist = np.zeros((N_elt, N_ast))

for idx in range(N_elt):
    dist[i] = tf.reduce_mean(tf.linalg.norm(X - Y[idx], axis=-1), axis=-1)

In [65]:
# Need to do this one candidate at a time b/c run out of memory when doing all at once
dist = np.zeros((N_elt, N_ast))

# Iterate over candidates
for idx in range(N_elt):
    dist[i] = tf.reduce_mean(tf.linalg.norm(X - Y[idx], axis=-1), axis=-1)



(32, 733489)

In [66]:
# Col number of nearest asteroid elements
col_idx = np.argmin(dist, axis=0)



(32, 733489)

In [ ]:
# Distance to nearest asteroid element
row_idx = np.arange(row_idx.size, dtype=np.int32)
dist_au = dist[row_idx, col_idx]